In [ ]:
!pip install streamlit opencv-python-headless tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.3 MB/s eta 0:00:00


In [ ]:
with open('deepfake_detect.py', 'w') as f:
  f.write('''import streamlit as st
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input

# Load the pre-trained EfficientNetB0 model and add a custom output layer
base_model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')
x = base_model.output
x = Dense(1, activation='sigmoid')(x)  # Sigmoid activation for binary classification
model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Function to preprocess images
def preprocess_image(image):
    image = cv2.resize(image, (224, 224))  # Resize to model input size
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    return image

# Function to predict if the image is real or fake
def predict_image(image):
    processed_image = preprocess_image(image)
    prediction = model.predict(processed_image)
    confidence_score = prediction[0][0]  # Get the confidence score
    return confidence_score  # Return the confidence score

# Function to process video and predict frame-wise
def predict_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_predictions = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        processed_frame = preprocess_image(frame)
        prediction = model.predict(processed_frame)
        frame_predictions.append(prediction[0][0])

    cap.release()

    # Aggregate results
    avg_prediction = np.mean(frame_predictions)
    return avg_prediction

# Streamlit user interface
st.title("Deepfake Detection")
st.write("Upload an image or video to check if it's real or fake.")

input_type = st.selectbox("Select input type:", ["Image", "Video"])

if input_type == "Image":
    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "png"])
    if uploaded_file is not None:
        image = cv2.imdecode(np.frombuffer(uploaded_file.read(), np.uint8), cv2.IMREAD_COLOR)
        st.image(image, channels="BGR")
        prediction = predict_image(image)
        prediction_label = "Fake" if prediction > 0.5 else "Real"
        st.write(f"Prediction: {prediction_label} with confidence: {prediction:.2f}")

elif input_type == "Video":
    uploaded_file = st.file_uploader("Choose a video...", type=["mp4", "avi"])
    if uploaded_file is not None:
        video_path = uploaded_file.name
        with open(video_path, "wb") as f:
            f.write(uploaded_file.getbuffer())

        avg_prediction = predict_video(video_path)
        avg_prediction_label = "Fake" if avg_prediction > 0.5 else "Real"
        st.write(f"Average Prediction: {avg_prediction_label} with confidence: {avg_prediction:.2f}")

# Run the Streamlit app
if __name__ == "__main__":
    pass''')


In [ ]:
!pip install pyngrok

In [ ]:
!streamlit run deepfake_detect.py &




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.145.135.65:8501

2025-03-08 20:05:16.403043: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741464316.430260    1364 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741464316.438954    1364 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-08 20:05:16.468103: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebui

In [ ]:
from pyngrok import ngrok

# Set your ngrok auth token (if required)
ngrok.set_auth_token("2u2g0bVsaLX2Y2uXI4mGTRewcyl_7L66yuj3KNzgr86evdHAf")  # Replace with your actual token if you have one
public_url = ngrok.connect(8501)  # Connect to the port Streamlit is using
print("Your Streamlit app is accessible at:", public_url)

Your Streamlit app is accessible at: NgrokTunnel: "https://46e2-34-145-135-65.ngrok-free.app" -> "http://localhost:8501"
